# kNN как классификатор

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
path = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
headernames = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'Class']
dataset = pd.read_csv(path, names = headernames)
dataset.head()

,sepal-length,sepal-width,petal-length,petal-width,Class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
dataset['Class'].value_counts()

Class
Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: count, dtype: int64

In [4]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 4].values

Разделение на обучающую и тестовую выборки

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.40)

Далее, масштабирование данных будет сделано следующим образом:

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

Далее обучаем модель с помощью sklearn и класса KNeighborsClassifier

In [7]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5)
classifier.fit(X_train, y_train)

,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


Наконец нам нужно сделать прогноз

In [8]:
y_pred = classifier.predict(X_test)

результаты:

In [9]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
result = confusion_matrix(y_test, y_pred)
print("Матрица ошибок:")
print(result)
result1 = classification_report(y_test, y_pred)
print("Метрики:",)
print (result1)
result2 = classifier.score(X_test,y_test)
print("f1-score:",result2)

Матрица ошибок:
[[26  1  0]
 [ 0 15  1]
 [ 0  3 14]]
Метрики:
                 precision    recall  f1-score   support

    Iris-setosa       1.00      0.96      0.98        27
Iris-versicolor       0.79      0.94      0.86        16
 Iris-virginica       0.93      0.82      0.88        17

       accuracy                           0.92        60
      macro avg       0.91      0.91      0.90        60
   weighted avg       0.92      0.92      0.92        60

f1-score: 0.9166666666666666


# Классификация с визуализацией с помощью модуля plotly

In [10]:
import plotly
import plotly.graph_objects as go

import numpy as np

import sklearn
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

Библиотеки быстро эволюционируют, потому указывается рабочая версия, с которой работал автор на момент написания блокнота. Для быстрого дебаггинга используйте команду для установки явной версии '!pip install [библиотека]==[версия]’.

In [11]:
print(np.__version__)
print(plotly.__version__)
print(sklearn.__version__)

2.2.6
6.5.0
1.7.2


In [12]:
# Переменная повлияет на сглаженность разделительных границ
mesh_size = .2

# Параметр повлияет на ширину каждой из зон на графике
margin = 0.25

# noise добавляет реалистичный шум
X, y = make_moons(noise = 0.3, random_state = 0)

# Разделим датасет на учебную и тестовую части
X_train, X_test, y_train, y_test = train_test_split(
    X, y.astype(str), test_size = 0.25, random_state = 0)

In [13]:
# X[:, 0].min() выберет из всех значений координат x минимальное значение
x_min, x_max = X[:, 0].min() - margin, X[:, 0].max() + margin
y_min, y_max = X[:, 1].min() - margin, X[:, 1].max() + margin

# np.arange создаст вектор, где каждое следующее значение равноудалено
# от предыдущего (шаг – mesh_size). Это ляжет в основу шкал на осях.
xrange = np.arange(x_min, x_max, mesh_size)
yrange = np.arange(y_min, y_max, mesh_size)

# np.meshgrid() создаст координатную матрицу
xx, yy = np.meshgrid(xrange, yrange)

# Создадим классификатор и классифицируем тренировочные данные
# weights = 'uniform' означает, что у всех точек
# одинаковое влияние на границы классификации
clf = KNeighborsClassifier(5, weights = 'uniform')
clf.fit(X, y)

# Ось z – это непосредственно предсказания
# np.c_ соединит последовательности координат в столбец
# ravel() сделает из этого столбца одномерный массив
Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
Z = Z.reshape(xx.shape)

In [15]:
# Обозначим учебные и тренировочные данные разными маркерами для наглядности
trace_specs = [
    [X_train, y_train, '0', 'Train', 'square'],
    [X_train, y_train, '1', 'Train', 'circle'],
    [X_test, y_test, '0', 'Test', 'square-dot'],
    [X_test, y_test, '1', 'Test', 'circle-dot']
]

fig = go.Figure(data = [
    go.Scatter( # специальная обертка Plotly: наслоим для наглядности "луны"
        x = X[y == label, 0], y = X[y == label, 1],
        # Настроим легенду графика
        name = f'{split} Split, Label {label}',
        mode = 'markers', marker_symbol = marker
    )
    for X, y, label, split, marker in trace_specs
])

# Зададим размер маркера, толщину обводки и цвет
fig.update_traces(
    marker_size = 12, marker_line_width = 1.5,
    marker_color = "white"
)

fig.add_trace(
    go.Contour( # Теперь отобразим саму карту "уверенности"
        x = xrange, # Шкалы размечаются с помощью равноудаленных точек
        y = yrange,
        z = Z, # Третье измерение – это цвет как степень уверенности
        showscale = False,
        colorscale = 'aggrnyl', # Предустановленная цветовая схема
        opacity = 1,
        name = 'Score',
        hoverinfo = 'skip'
    )
)
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

#kNN в качестве регрессора

In [ ]:
data = pd.read_csv(path, names = headernames)
array = data.values
X = array[:,:2]
y = array[:,2]

In [ ]:
X

array([[5.1, 3.5],
       [4.9, 3.0],
       [4.7, 3.2],
       [4.6, 3.1],
       [5.0, 3.6],
       [5.4, 3.9],
       [4.6, 3.4],
       [5.0, 3.4],
       [4.4, 2.9],
       [4.9, 3.1],
       [5.4, 3.7],
       [4.8, 3.4],
       [4.8, 3.0],
       [4.3, 3.0],
       [5.8, 4.0],
       [5.7, 4.4],
       [5.4, 3.9],
       [5.1, 3.5],
       [5.7, 3.8],
       [5.1, 3.8],
       [5.4, 3.4],
       [5.1, 3.7],
       [4.6, 3.6],
       [5.1, 3.3],
       [4.8, 3.4],
       [5.0, 3.0],
       [5.0, 3.4],
       [5.2, 3.5],
       [5.2, 3.4],
       [4.7, 3.2],
       [4.8, 3.1],
       [5.4, 3.4],
       [5.2, 4.1],
       [5.5, 4.2],
       [4.9, 3.1],
       [5.0, 3.2],
       [5.5, 3.5],
       [4.9, 3.1],
       [4.4, 3.0],
       [5.1, 3.4],
       [5.0, 3.5],
       [4.5, 2.3],
       [4.4, 3.2],
       [5.0, 3.5],
       [5.1, 3.8],
       [4.8, 3.0],
       [5.1, 3.8],
       [4.6, 3.2],
       [5.3, 3.7],
       [5.0, 3.3],
       [7.0, 3.2],
       [6.4, 3.2],
       [6.9,

In [ ]:
y

array([1.4, 1.4, 1.3, 1.5, 1.4, 1.7, 1.4, 1.5, 1.4, 1.5, 1.5, 1.6, 1.4,
       1.1, 1.2, 1.5, 1.3, 1.4, 1.7, 1.5, 1.7, 1.5, 1.0, 1.7, 1.9, 1.6,
       1.6, 1.5, 1.4, 1.6, 1.6, 1.5, 1.5, 1.4, 1.5, 1.2, 1.3, 1.5, 1.3,
       1.5, 1.3, 1.3, 1.3, 1.6, 1.9, 1.4, 1.6, 1.4, 1.5, 1.4, 4.7, 4.5,
       4.9, 4.0, 4.6, 4.5, 4.7, 3.3, 4.6, 3.9, 3.5, 4.2, 4.0, 4.7, 3.6,
       4.4, 4.5, 4.1, 4.5, 3.9, 4.8, 4.0, 4.9, 4.7, 4.3, 4.4, 4.8, 5.0,
       4.5, 3.5, 3.8, 3.7, 3.9, 5.1, 4.5, 4.5, 4.7, 4.4, 4.1, 4.0, 4.4,
       4.6, 4.0, 3.3, 4.2, 4.2, 4.2, 4.3, 3.0, 4.1, 6.0, 5.1, 5.9, 5.6,
       5.8, 6.6, 4.5, 6.3, 5.8, 6.1, 5.1, 5.3, 5.5, 5.0, 5.1, 5.3, 5.5,
       6.7, 6.9, 5.0, 5.7, 4.9, 6.7, 4.9, 5.7, 6.0, 4.8, 4.9, 5.6, 5.8,
       6.1, 6.4, 5.6, 5.1, 5.6, 6.1, 5.6, 5.5, 4.8, 5.4, 5.6, 5.1, 5.1,
       5.9, 5.7, 5.2, 5.0, 5.2, 5.4, 5.1], dtype=object)

Импортируем KNeighborsRegressor из sklearn

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knnr = KNeighborsRegressor(n_neighbors = 5)
knnr.fit(X, y)

KNeighborsRegressor()

Мы можем найти RMSE следующим образом

In [ ]:
print ("The RMSE is:",format(np.sqrt(np.power(y-knnr.predict(X),2).mean())))

The RMSE is: 0.3813379953094978
